<b>TASK 1A

In [60]:
#TASK 1A

import math
import numpy as np
import pandas as pd

class DecisionTree:
    def __init__(self, stopping_depth=None):
        self.decision_tree = None
        self.stopping_depth = stopping_depth
        self.depths_reached = []
        self.max_depth_reached = None

    def information_gain(self, feature_matrix, label_data, feature, condition):
        entropy_before = self.entropy(label_data)
        feature_data = feature_matrix[feature]

        cond_sat_labels = label_data[feature_data == condition] # Labels for satisfied condition
        cond_not_sat_labels = label_data[feature_data != condition] # Labels for not satisfied condition

        # Calculating entropy after splitting on the feature's condition
        num_cond_sat_labels = len(cond_sat_labels)
        num_cond_not_sat_labels = len(cond_not_sat_labels)
        num_labels = len(label_data)
        entropy_after = ((num_cond_sat_labels / num_labels) * self.entropy(cond_sat_labels)) + \
        ((num_cond_not_sat_labels / num_labels) * self.entropy(cond_not_sat_labels))
        
        # Calculating information gain
        information_gain = entropy_before - entropy_after
        
        return information_gain

    def entropy(self, label_data):
        labels, label_counts = np.unique(label_data, return_counts=True) # Get labels (0,1) and counts of each
        total_count = sum(label_counts)
        probabilities = []

        i = 0
        for label in labels:
            probabilities.append(label_counts[i] / total_count)
            i+=1
        
        entropy = 0
        for p in probabilities:
            if p != 0:
                entropy += ((-p) * math.log2(p)) # make sure not to do log2(0)

        return entropy
    
    def find_best_split(self, feature_matrix, label_data):
        best_condition = None # Initialise best condition of best feature to split on
        highest_info_gain = 0 # Initialise highest information gain
        features = feature_matrix.columns # Get names of all features
        best_feature = None # Initialise best feature to split on

        # Iterate over all features to find best split (highest information gain)
        for feature in features:
            categories = np.unique(feature_matrix[feature]) # Get feature's categories
            for category in categories:
                info_gain = self.information_gain(feature_matrix, label_data, feature, \
                                                  category)
                if info_gain > highest_info_gain:
                    best_condition = category
                    highest_info_gain = info_gain
                    best_feature = feature

        return best_feature, best_condition, highest_info_gain     

    # recursively train decision tree
    def train(self, feature_matrix, label_data, current_depth=0):
        self.depths_reached.append(current_depth)
        self.max_depth_reached = max(self.depths_reached)
        labels = np.unique(label_data) # Get labels (0,1)
        
        # Base case 1: If there is only one label type, return that label
        num_labels = len(labels)
        if num_labels == 1:
            return labels[0].item()
            
        # Base case 2: There are no more features to split on or we reached stopping depth
        if feature_matrix.empty or (self.stopping_depth is not None and current_depth >= self.stopping_depth):
            if label_data.empty:
                return 0
            else:
                return label_data.mode()[0] # return majority class

        best_feature, best_condition, highest_info_gain = self.find_best_split(feature_matrix, \
                                                                       label_data)
        # If there is no split that can achieve information gain return majority class
        if highest_info_gain == 0:
            if label_data.empty:
                return 0
            else:
                return label_data.mode()[0]

        # Recursive case:
        best_feature_data = feature_matrix[best_feature] # Get data of best feature
        # Get matrix rows where best feature has best condition
        right_feature_matrix = feature_matrix[best_feature_data == best_condition]
        right_feature_matrix = right_feature_matrix.drop(columns=[best_feature]) #drop best_feature column as we use it to split
        # Get matrix rows where best feature doesn't have best condition
        left_feature_matrix = feature_matrix[best_feature_data != best_condition]
        left_feature_matrix = left_feature_matrix.drop(columns=[best_feature]) #drop best_feature column as we use it to split
        # Get label data where best feature has best condition
        right_label_data = label_data[best_feature_data == best_condition]
        # Get label data where best feature doesn't have best condition
        left_label_data = label_data[best_feature_data != best_condition]

        # Create dictionary for feature
        decision_tree_dict = {best_feature: {}}
        # In 'best_condition' key put the right of the tree
        decision_tree_dict[best_feature][best_condition] = self.train(right_feature_matrix, right_label_data, current_depth+1)
        # In 'not best_condition' key put the left of the tree
        decision_tree_dict[best_feature][f"not {best_condition}"] = self.train(left_feature_matrix, left_label_data, current_depth+1)

        self.decision_tree = decision_tree_dict
        return decision_tree_dict

    # Reursively predict label for a single row
    def predict_label(self, row, decision_subtree):
        # Base case: if the decision tree is not a dictionary it is a leaf, return the label
        if not isinstance(decision_subtree, dict):
            return decision_subtree

        first_feature = list(decision_subtree.keys())[0] # Convert to list so we can get first key
        row_first_feature_value = row.get(first_feature, None) # Get value of root feature in row

        # If the row's value for the first feature exists in tree take that path
        if row_first_feature_value in decision_subtree[first_feature]:
            return self.predict_label(row, decision_subtree[first_feature][row_first_feature_value])

        # If the row's value for the first feature does not exist in the tree take the "not" path
        first_feature_subtree = decision_subtree[first_feature].items()
        for condition, branches in first_feature_subtree:
            if condition.startswith("not"):
                return self.predict_label(row, branches)

        # If there is no match (shouldn't happen)
        return 0

    # Use predict_label function to predict for each row in feature matrix
    def predict_all_labels(self, feature_matrix):
        label_predictions = []
        
        for i, row in feature_matrix.iterrows():
            label_prediction = self.predict_label(row, self.decision_tree)
            label_predictions.append(label_prediction)

        predictions_df = pd.Series(label_predictions, index=feature_matrix.index)
        return predictions_df

    # Recursively print decision tree
    def print_decision_tree(self, decision_tree_dict, indent=""):
        # Base case: we are at the leaf (so it's not a dict)
        if not isinstance(decision_tree_dict, dict):
            print(indent,"Leaf:",decision_tree_dict)
            return

        # Recursive case
        decision_tree = decision_tree_dict.items()
        for feature, branches in decision_tree:
            print(indent,"Feature:",feature)

            subtree = branches.items()
            for condition, condition_branches in subtree:
                if condition.startswith("not"):
                    print(indent[0:-4],"Condition:",condition)
                else:
                    print(indent,"Condition:",condition)
                indent += "    "
                self.print_decision_tree(condition_branches, indent)
            

# Load train data
training_data = pd.read_csv("adult_train_data.csv")

# Split up train data
training_label_column = training_data.columns[-1] # Last column (just the title)
training_feature_matrix = training_data.drop(columns=[training_label_column]) # Get all columns except label column
training_label_data = training_data[training_label_column] # Get data for label column

# Train official tree (no specified stopping_depth)
training_tree = DecisionTree()
trained_tree = training_tree.train(training_feature_matrix, training_label_data)
training_tree.print_decision_tree(trained_tree)
print("Max depth reached:", training_tree.max_depth_reached)


 Feature: marital_status
 Condition: Married-civ-spouse
     Feature: education_num
     Condition: >=10
         Feature: education
         Condition: Some-college
             Feature: age
             Condition: 20-29
                 Feature: hours_per_week
                 Condition: <35
                     Feature: fnlwgt
                     Condition: 200K-300K
                         Feature: occupation
                         Condition: Adm-clerical
                             Feature: relationship
                             Condition: Husband
                                 Leaf: 0
                             Condition: not Husband
                                     Leaf: 1
                         Condition: not Adm-clerical
                                 Leaf: 0
                     Condition: not 200K-300K
                             Leaf: 0
                 Condition: not <35
                         Feature: occupation
                         Condition: O

<b>TASK 1B

In [61]:
# TASK 1B

# Train example trees with stopping_depth 2,3,4
for ex_stopping_depth in [2,3,4]:
    ex_tree = DecisionTree(ex_stopping_depth)
    trained_ex_tree = ex_tree.train(feature_matrix, label_data)
    print("Decision tree with stopping depth {}:\n".format(ex_stopping_depth))
    ex_tree.print_decision_tree(trained_ex_tree)
    print("\nMax depth reached:", ex_tree.max_depth_reached)
    print("\n")

Decision tree with stopping depth 2:

 Feature: marital_status
 Condition: Married-civ-spouse
     Feature: education_num
     Condition: >=10
         Leaf: 1
     Condition: not >=10
             Leaf: 0
 Condition: not Married-civ-spouse
         Feature: hours_per_week
         Condition: >40
             Leaf: 0
         Condition: not >40
                 Leaf: 0

Max depth reached: 2


Decision tree with stopping depth 3:

 Feature: marital_status
 Condition: Married-civ-spouse
     Feature: education_num
     Condition: >=10
         Feature: education
         Condition: Some-college
             Leaf: 0
         Condition: not Some-college
                 Leaf: 1
     Condition: not >=10
             Feature: education
             Condition: HS-grad
                 Leaf: 0
             Condition: not HS-grad
                     Leaf: 0
 Condition: not Married-civ-spouse
         Feature: hours_per_week
         Condition: >40
             Feature: education_num
          

For all three trees of depths 2,3,4 they have the same initial splits but successive trees have one extra level of depth. The tree of depth 2 has very few splits and risks underfitting and high bias while the deeper trees are more likely to have lower bias, but higher variance as we could be overfitting to our training data.


<b>TASK 1C

In [62]:
# TASK 1C

# Training
training_predictions = training_tree.predict_all_labels(training_feature_matrix)

# Evaluate training predictions
train_tp = ((training_predictions == training_label_data) & (training_predictions == 1)).sum() # TP
train_tn = ((training_predictions == training_label_data) & (training_predictions == 0)).sum() # TN
train_tp_tn = (training_predictions == training_label_data).sum() # TP + TN
train_fp = ((training_predictions == 1) & (training_label_data == 0)).sum() # FP
train_fn = ((training_predictions == 0) & (training_label_data == 1)).sum() # FN
train_num_labels = len(training_label_data) # Total

train_accuracy = train_tp_tn/train_num_labels
train_precision = train_tp/(train_tp+train_fp)
train_recall = train_tp/(train_tp+train_fn)

# Testing
testing_data = pd.read_csv("adult_test_data.csv") # Testing data
testing_label_column = testing_data.columns[-1] # Last column (just the title)
testing_feature_matrix = testing_data.drop(columns=[testing_label_column]) # Get all columns except label column
testing_label_data = testing_data[testing_label_column] # Get data for label column
testing_predictions = training_tree.predict_all_labels(testing_feature_matrix)

# Evaluate testing predictions
test_tp = ((testing_predictions == testing_label_data) & (testing_predictions == 1)).sum() # TP
test_tn = ((testing_predictions == testing_label_data) & (testing_predictions == 0)).sum() # TN
test_tp_tn = (testing_predictions == testing_label_data).sum() # TP + TN
test_fp = ((testing_predictions == 1) & (testing_label_data == 0)).sum() # FP
test_fn = ((testing_predictions == 0) & (testing_label_data == 1)).sum() # FN
test_num_labels = len(testing_label_data) # Total

test_accuracy = test_tp_tn/test_num_labels
test_precision = test_tp/(test_tp+test_fp)
test_recall = test_tp/(test_tp+test_fn)

print("Label predictions for test data:")
print(training_predictions)

print("\nTrain accuracy:", train_accuracy)
print("Train precision:", train_precision)
print("Train recall:", train_recall)

print("\nTest accuracy:", test_accuracy)
print("Test precision:", test_precision)
print("Test recall:", test_recall)

approx_error = (1.0-test_accuracy) - (1.0-train_accuracy)
print("\nApproximation error:",approx_error)

Label predictions for test data:
0        0
1        1
2        0
3        0
4        0
        ..
30156    1
30157    0
30158    0
30159    0
30160    1
Length: 30161, dtype: int64

Train accuracy: 0.8343224694141441
Train precision: 0.7065306793880572
Train recall: 0.5720564730953649

Test accuracy: 0.8207835325365206
Test precision: 0.668235294117647
Test recall: 0.5372972972972972

Approximation error: 0.01353893687762353


The train accuracy is high (>>50%) and very similar to the test accuracy so the approximation error is very low. This means the predictions my decision tree (that I trained with the training data) makes on both the training and testing data are fairly accurate compared to the true labels. The training precision and recall are both a bit higher than for the test but all are greater than 50%. 

<b>TASK 2

<b>A. Discuss what will happen if you decide to change the splitting criterion. Explain the new splitting criterion and how it might change your decision tree.

Right now we are splitting according to the information gain criterion which is based on entropy. However, we can use different criterion such as 'Gain Ratio'. The problem with information gain as a splitting criterion arises when features have lots of possible categories because when we create a split with high information gain the partitions can become very small. Gain ratio, however, considers how many possible categories a feature can have to ensure our tree does not end up having as many small branches as it will penalise those features with many categories. Gain ratio is calculated by dividing the information gain by the split information (the higher the number of categories the higher the split information and thus the lower the gain ratio). Using this criterion will most likely result in a tree that is not as deep.


<b>B. Explain whether your test procedure implemented in Task 1-C can indicate whether your tree is over- or underfitting.  

The difference between my training accuracy and my testing accuracy is very low, which means my model has a low approximation error. This means my trained decision tree is barely overfitting. Since my training accuracy is slightly higher than testing accuracy we can say there is a tiny bit of overfitting, meaning my splits became slightly too specific for the training data.